In [1]:
"""
데커레이터?
- 다른 함수를 인수로 받는 콜러블, 함수나 메서드에 적용되어 기능을 추가, 확장, 변경하는 역할
- 데커레이트된 함수에 어떤 처리를 수행, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체함 
- 데커레이트된 함수를 다른 함수로 대체함
- 데커레이터는 모듈이 로딩될 때 바로 실행됨
"""

'\n데커레이터?\n- 다른 함수를 인수로 받는 콜러블\n- 데커레이트된 함수에 어떤 처리를 수행, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체함 \n- 데커레이트된 함수를 다른 함수로 대체함\n- 데커레이터는 모듈이 로딩될 때 바로 실행됨\n'

In [5]:
"""
러타임에 프로그램 행위를 변경하는 메타프로그래밍을 할 때 데커레이터가 상당히 편리함
"""

def deco(func) : 
    def inner() : 
        print('running inner()')
    return inner

@deco
def target() :
    print('running target()')
    
target()

running inner()


In [6]:
target

<function __main__.deco.<locals>.inner()>

In [7]:
"""
데커레이터는 임포트 타임에 실행됨(ㅇ파이썬이 모듈을 로딩하는 시점)
- 함수 데커레이터는 모듈이 임포트되자마자 실행
- 데커레이트된 함수는 명시적으로 호출될 때만 실행
"""
registry = []

def register(func) : 
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1() :
    print('running f1()')
    
@register
def f2() : 
    print('running f2()')
    
def f3() : 
    print('running f3()')
    
def main() : 
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__' : 
    main()
    

running register(<function f1 at 0x7fe1e0a6e820>)
running register(<function f2 at 0x7fe1e0ae53a0>)
running main()
registry -> [<function f1 at 0x7fe1e0a6e820>, <function f2 at 0x7fe1e0ae53a0>]
running f1()
running f2()
running f3()


In [8]:
"""
데커레이터로 개선한 전략 패턴 

"""
promos = []

def promotion(promo_func) : 
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order) : 
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
@promotion
def bulk_item(order) :
    discount = 0
    
    for i in order.cart : 
        if i.quantity >= 20 : 
            disocunt += i.total() * .1
            
    return discount

@promotion
def large_order(order) : 
    distinct_items = {i.product for i in order.cart}
    
    if len(distinct_items) >= 10 : 
        return order.total() * .07
    
    return 0 

def best_promo(order) : 
    return max(promo(order) for promo in promos)


In [9]:
"""
파이썬이 함수 본체를 컴파일할 때 b가 함수 안에서 할당되므로 b를 지역 변수로 판단
"""

b = 10

def f2(a) : 
    print(a)
    print(b)
    
    b = 1000

f2(10)

10


UnboundLocalError: local variable 'b' referenced before assignment

In [11]:
b = 10

def f2(a) : 
    global b
    print(a)
    print(b)
    
    b = 1000

f2(10)

10
10


In [13]:
"""
클로저?
- 함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수
- 함수가 익명 함수인지 여부는 중요치 않음, 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요함
- 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수 
- 즉, 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있는 것을 의미함

"""

class Averager() : 
    
    def __init__(self) : 
        self.series = []
        
    def __call__(self, new_value) :
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg1 = Averager()
avg1(10)

10.0

In [14]:
avg1(11)

10.5

In [15]:
avg1(12)

11.0

In [17]:
"""
- make_averager()는 average() 함수 객체를 반환
- 호출될 때 마다 받은 인수를 저장하고 현재까지의 평균을 계산함 
"""
def make_averager() : 
    # 클로저 
    series = []
    
    def averager(new_value) : 
        # series는 자유 변수(지역 범위에 바인딩되지 않은 변수)
        series.append(new_value)
        total = sum(series)
        
        return total / len(series)
    
    return averager

avg2 = make_averager()
avg2(10)

10.0

In [18]:
avg2(11)

10.5

In [19]:
avg2(12)

11.0

In [20]:
avg2.__code__.co_varnames

('new_value', 'total')

In [21]:
avg2.__code__.co_freevars

('series',)

In [22]:
avg2.__closure__

(<cell at 0x7fe1d1be10a0: list object at 0x7fe1d26a7180>,)

In [23]:
"""
nonlocal 사용
"""

def make_averager() : 
    count = 0
    total = 0
    
    def averager(new_value) : 
        nonlocal count, total
        
        count += 1
        total += new_value
        
        return total / count
    
    return averager

In [25]:
import time


"""
함수의 실행 시간을 출력하는 간단한 데커레이터 
"""
def clock(func) : 
    def clocked(*args) : 
        t0 = time.perf_counter() 
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


@clock
def snooze(seconds) : 
    time.sleep(seconds)
    
@clock
def factorial(n) : 
    return 1 if n < 2 else n * factorial(n-1)

if __name__ == '__main__' : 
    print('*' * 40, "Calling snooze(.123)")
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! = ', factorial(6))


**************************************** Calling snooze(.123)
[0.12806692s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000129s] factorial(1) -> 1
[0.00006675s] factorial(2) -> 2
[0.00011946s] factorial(3) -> 6
[0.00017146s] factorial(4) -> 24
[0.00022104s] factorial(5) -> 120
[0.00027300s] factorial(6) -> 720
6! =  720


In [36]:
import time, functools

def clock(func) : 
    @functools.wraps(func)
    def clocked(*args, **kwargs) : 
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = [] 
        
        if args : 
            arg_lst.append(', '.join(repr(arg) for arg in args))
        
        if kwargs : 
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
            
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked



In [37]:
"""
파이썬에서는 메서드를 데코레이트하기 위해 3가지 내장 함수 제공  
- property(), classmethod(), staticmethod()
"""

'\n파이썬에서는 메서드를 데코레이트하기 위해 3가지 내장 함수 제공  \n- property(), classmethod(), staticmethod()\n'

In [38]:
"""
캐시를 이용한 성능 좋은 데커레이터 구현
"""

import functools

@functools.lru_cache()
@clock
def fibonacci(n) : 
    if n < 2 : 
        return n
    
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == '__main__' : 
    print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0 
[0.00000095s] fibonacci(1) -> 1 
[0.00044322s] fibonacci(2) -> 1 
[0.00000215s] fibonacci(3) -> 2 
[0.00069308s] fibonacci(4) -> 3 
[0.00000072s] fibonacci(5) -> 5 
[0.00075698s] fibonacci(6) -> 8 
8


In [39]:

from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch  # <1>
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register  # <2>
def _(text: str) -> str:  # <3>
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register  # <4>
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register  # <5>
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register  # <6>
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)  # <7>
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)  # <8>
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

In [42]:
"""
누적된 데코레이터 처리 방식
"""

def d1(func) : 
    pass

def d2(func) : 
    pass

@d1
@d2
def f() : 
    print('f')
    
# 위에 코드는 밑에와 같이 움직임
def f() : 
    print('f')
    
f = d1(d2(f))

In [43]:
"""
매개변수화된 등록 데커레이터
"""

registry = set()  # <1>

def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register'
              f'(active={active})->decorate({func})')
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>

@register(active=False)  # <8>
def f1():
    print('running f1()')

@register()  # <9>
def f2():
    print('running f2()')

def f3():
    print('running f3()')

# end::REGISTRATION_PARAM[]

running register(active=False)->decorate(<function f1 at 0x7fe1e0b47820>)
running register(active=True)->decorate(<function f2 at 0x7fe1e0a6ed30>)


In [44]:
registry

{<function __main__.f2()>}

In [46]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7fe1b002a820>)


<function __main__.f3()>

In [47]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [48]:
register(active = False)(f2)

running register(active=False)->decorate(<function f2 at 0x7fe1e0a6ed30>)


<function __main__.f2()>

In [49]:
registry

{<function __main__.f3()>}

In [57]:
"""
매개변수화된 clock 데커레이터
"""

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT) : 
    def decorate(func) : 
        def clocked(*_args) : 
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

if __name__ == '__main__' : 
    
    @clock() 
    def snooze(seconds) : 
        time.sleep(seconds)
        
    for i in range(3) :
         snooze(.123)
            

[0.12808084s] snooze(0.123) -> None
[0.12806201s] snooze(0.123) -> None
[0.12808704s] snooze(0.123) -> None
